# Mediation

In [24]:
from pymer4 import load_dataset
from pymer4.models import lm, lmer

cars = load_dataset("mtcars")
cars.head()

model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
str,f64,i64,f64,i64,f64,f64,f64,i64,i64,i64,i64
"""Mazda RX4""",21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4
"""Mazda RX4 Wag""",21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4
"""Datsun 710""",22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
"""Hornet 4 Drive""",21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
"""Hornet Sportabout""",18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2


In [12]:
m1 = lm("hp ~ wt", data=cars)
m1.fit()

m2 = lm("mpg ~ wt + hp", data=cars)
m2.fit()

In [22]:
from rpy2.robjects.packages import importr
import pymer4.tidystats as ts
mediation = importr("mediation")
broom = importr("broom")

tidy_mediate = broom.tidy_mediate
summary = lambda x: print(ts.summary(x))

out = mediation.mediate(m1.r_model, m2.r_model, treat="wt", mediator="hp")
tibble = tidy_mediate(out)

In [21]:
ts.R2polars(tibble)

term,estimate,std_error,p_value
str,f64,f64,f64
"""acme_0""",-1.446535,0.50501,0.002
"""acme_1""",-1.446535,0.50501,0.002
"""ade_0""",-3.909733,0.649161,0.0
"""ade_1""",-3.909733,0.649161,0.0


In [23]:
summary(out)



Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value    
ACME             -1.443       -2.583        -0.56  <2e-16 ***
ADE              -3.890       -5.099        -2.72  <2e-16 ***
Total Effect     -5.333       -6.380        -4.27  <2e-16 ***
Prop. Mediated    0.268        0.107         0.46  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Sample Size Used: 32 


Simulations: 1000 




In [30]:
m3 = lmer("mpg ~ wt + hp + (1 | cyl)", data=cars)
m3.fit()

m4 = lmer("hp ~ wt + (1 | cyl)", data=cars)
m4.fit()

summary(mediation.mediate(m3.r_model, m4.r_model, treat="wt", mediator="hp", group_out="cyl"))

RRuntimeError: Error in (function (model.m, model.y, sims = 1000, boot = FALSE, boot.ci.type = "perc",  : 
  mediator model is not yet implemented
